<br><br>
<h1 style="font-size:36px" align="center"> Cluster and remove highly similar sequences</h1><br><br><br><br><br><br>

In [1]:
import os
import glob
from Bio import SeqIO
from Bio import AlignIO
from tqdm.auto import tqdm
import pylev
import matplotlib.pyplot as plt
import numpy as np
import copy
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
import csv

def remove_gaps(seq_record):
    seq = seq_record.seq
    seq_without_gaps = Seq("".join(str(seq).split("-")))
    return SeqRecord(seq_without_gaps, id=seq_record.id, description=seq_record.description)


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h3 style="font-size:24px"> Define Parameters</h3><br>

In [2]:
cluster_min_size = 5
skip_cluster = False
overwrite_cluster = False
ID_cutoff = 0.9
input_MSA_filename = "../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_MSA.fa"
dataset_name = "clustalo"

work_dir = "../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/"
cluster_script_filename = "../../external_scripts/ClusterMSA.py"

if not os.path.exists(work_dir):
    os.makedirs(work_dir)
if not skip_cluster:
    cluster_str = f"{cluster_min_size}_per_cluster"
else:
    cluster_str = "no_cluster"
cluster_dir = os.path.join(work_dir,dataset_name +f"_{cluster_str}_clusters")
output_filename = os.path.join(work_dir, f"{dataset_name}_{cluster_str}_{int(ID_cutoff*100)}_ID_initial_MSA.fa")
cdhit_input_filename = os.path.join(cluster_dir,"clustered_sequences.fa")
cdhit_output_filename = os.path.join(cluster_dir,"cdhit_output.fa")
tree_data_output_filename = os.path.join(work_dir, f"{dataset_name}_{cluster_str}_{int(ID_cutoff*100)}_ID_tree_data.tsv")
if not os.path.exists(cluster_dir):
    os.makedirs(cluster_dir)


print(f"Creating Filtered Dataset for phylogenetic analysis '{output_filename}'")

Creating Filtered Dataset for phylogenetic analysis '../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_5_per_cluster_90_ID_initial_MSA.fa'


In [3]:
focused_sequences = [seqrec for seqrec in SeqIO.parse(input_MSA_filename,"fasta")]
for seq in focused_sequences:
    seq_id = seq.id.split(".")[0]
    if(seq_id != seq.id):
        raise Exception("Names are in the wrong format... check MSA, might need to be rerun")

In [4]:
clustered_sequences = []

print(f"Clustering sequences, and filtering out sequences with fewer than {cluster_min_size} sequences\n")
cluster_script = f"python {cluster_script_filename} clst -i {input_MSA_filename} -o {cluster_dir} --gap_cutoff 1.0 --min_samples {cluster_min_size}"
print(cluster_script)
if(not skip_cluster):
    if any(filename.endswith(".a3m") for filename in os.listdir(cluster_dir)):
        print(f"Previous instance of cluster detected in {cluster_dir}\n")
        if overwrite_cluster:
            print(f"Overwriting all files in {cluster_dir}")
            !rm -rf {cluster_dir}
            os.makedirs(cluster_dir)
            !$cluster_script
        else:
            print("Preserving previously created files, skipping clustering step")
    else:
        print("Initiating clustering.")
        print(f"Clustering sequences in {cluster_dir}")
        !$cluster_script

    import re
    pattern = re.compile(r"clst_\d{3}\.a3m")
    for filename in os.listdir(cluster_dir):
        if pattern.match(filename):
            file_path = os.path.join(cluster_dir, filename)
            seqs = [seqrec for seqrec in SeqIO.parse(file_path,"fasta")]
            if(len(seqs)>cluster_min_size):
                clustered_sequences += seqs
else:
    print("Skipping clustering step as requested, if you do not want this, please set skip_cluster to False!")
    clustered_sequences = copy.deepcopy(focused_sequences)
    
print(f"\nFiltering out sequences without large clusters.\nOut of {len(focused_sequences)} sequences, {len(clustered_sequences)} remain.")
focused_sequences = copy.deepcopy(clustered_sequences)


Clustering sequences, and filtering out sequences with fewer than 5 sequences

python ../../external_scripts/ClusterMSA.py clst -i ../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_MSA.fa -o ../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_5_per_cluster_clusters --gap_cutoff 1.0 --min_samples 5
Previous instance of cluster detected in ../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_5_per_cluster_clusters

Preserving previously created files, skipping clustering step

Filtering out sequences without large clusters.
Out of 6248 sequences, 4681 remain.


In [5]:
ungapped_sequences = []
for seq in focused_sequences:
    ungapped_sequences.append(remove_gaps(seq))

SeqIO.write(ungapped_sequences, cdhit_input_filename, "fasta")

print(f"Filtering out sequences with >{ID_cutoff*100}% identity.\n")
!cd-hit -i $cdhit_input_filename -o $cdhit_output_filename -c $ID_cutoff -n 5 -d 0 -T 8 -M 16000

cdhit_accessions = [seqrec.id for seqrec in SeqIO.parse(cdhit_output_filename,"fasta")]

low_similarity_sequences = []
for accession in cdhit_accessions:
    for seq2 in focused_sequences:
        if(seq2.id == accession):
            low_similarity_sequences.append(seq2)
            break

if(len(low_similarity_sequences) != len(cdhit_accessions)):
    print(len(low_similarity_sequences), len(focused_sequences))
    raise Exception("Something has gone wrong, the filtered by id sequences don't map to the same sequences as clustered sequences.\n Have you run all previous code or overwritten files?")
    
print(f"\nFiltered out sequences with >{ID_cutoff*100}% identity.")
print(f"Out of {len(focused_sequences)} sequences, {len(low_similarity_sequences)} remain.")
focused_sequences = copy.deepcopy(low_similarity_sequences)

Filtering out sequences with >90.0% identity.

Program: CD-HIT, V4.8.1 (+OpenMP), Aug 20 2021, 08:39:56
Command: cd-hit -i
         ../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_5_per_cluster_clusters/clustered_sequences.fa
         -o
         ../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_5_per_cluster_clusters/cdhit_output.fa
         -c 0.9 -n 5 -d 0 -T 8 -M 16000

Started: Fri Feb 23 09:17:53 2024
                            Output                              
----------------------------------------------------------------
Actual number of CPUs to be used: 6

total seq: 4681
longest and shortest : 400 and 200
Total letters: 1468584
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 2M
Buffer          : 6 X 10M = 63M
Table           : 2 X 65M = 130M
Miscellaneous   : 0M
Total           : 196M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting en

In [6]:
SeqIO.write(focused_sequences, output_filename, "fasta")
print(f"\nSequences saved in {output_filename}")


Sequences saved in ../../../data/ASST_processed_sequences/23_02_24_Quick_Tree/clustalo_5_per_cluster_90_ID_initial_MSA.fa


<br><br>
<h1 style="font-size:36px" align="center">Tree data creation</h1><br><br><br><br><br><br>

In [ ]:
import os
import glob
from Bio import SeqIO
from Bio import AlignIO
from tqdm.auto import tqdm
import pylev
import matplotlib.pyplot as plt
import numpy as np
import copy
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
import csv

<h3 style="font-size:24px"> Define Parameters</h3><br>

In [ ]:
total_tree_data_filename = "../processed_sequences/initial_dataset/tree_data.json"
use_alternate_sequences = True
alternate_sequences_filename = "../processed_sequences/clustal_hmm_5_per_cluster_18022023/seqs11.txt"



if(use_alternate_sequences):
    tree_data_output_filename = os.path.join(os.path.dirname(alternate_sequences_filename),"tree_data.tsv")
    focused_sequences = [seqrec for seqrec in SeqIO.parse(alternate_sequences_filename,"fasta")]


In [ ]:
sequence_accessions = [seq.id for seq in focused_sequences]
sequence_accessions.sort()
with open(total_tree_data_filename) as json_file:
    data = json.load(json_file)

In [ ]:
headers = list(data[sequence_accessions[0]].keys())
tree_data = {}
tree_data["accessions"] = sequence_accessions
for header in headers:
    tree_data[header] = []
    if(header == "related_seq"):
        related_sequences = {}
        for accession in sequence_accessions:
            for related_seq in data[accession][header]:
                if(related_seq[1] not in related_sequences):
                    related_sequences[related_seq[1]] = (accession,related_seq[0],related_seq[1])
                else:
                    if(related_sequences[related_seq[1]][1]>related_seq[0]):
                        related_sequences[related_seq[1]] = (accession,related_seq[0],related_seq[1])
        tree_data[header] = ["_"]*len(sequence_accessions)
        for related_seq in list(related_sequences.keys()):
                index = sequence_accessions.index(related_sequences[related_seq][0])
                tree_data[header][index] = related_seq
    else:
        for accession in sequence_accessions:
            tree_data[header].append(data[accession][header])


In [ ]:
with open(tree_data_output_filename, "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    
    # Write header row
    writer.writerow(tree_data.keys())
    
    # Write values rows
    rows = zip(*tree_data.values())
    for row in rows:
        writer.writerow(row)